# PREPROCESSING DATA FROM DATASET THAT CONTAINS THE TRANSLATED SENTENCES

In [42]:
"""
Compiled: Brikena Liko
Date: May 8, 2025
This script will prepare this resource:
"The list of the most common words of English", which is useful to anotate the "rare" feature
Credits for the original non-filtered list to: Rachael Tatman, dataset downloaded from https://www.kaggle.com/datasets/rtatman/english-word-frequency"
The dataset contains the counts of the 333,333 most commonly-used single words on the English language web, as derived from the Google Web Trillion Word Corpus
Because it is a list of unique words, but we are operating at the lemma level, with this script we will:
- lemmatize the list,
- remove lemmas dublicates
- summ up the count of same lemmas
- calculate the "times per million"
- keep the non-rare band frequency as described in the paper & instructions
"""

'\nThis will prepare these resource:\n"The list of the most common words of English" [useful for the "rare" feature]\nCredits for the original non-filtered list to:\n'

In [15]:
# !pip install stanza
# !pip install stanza pandas openpyxl

In [6]:
import stanza
import re
import pandas as pd
import numpy as np

In [11]:
#download the English model if not already installed
stanza.download('en')
#load the Stanza NLP pipeline
nlp = stanza.Pipeline(lang='en', processors="tokenize,lemma")


### Download the file from the following link, save in the same folder as script or change the file path when necessary
https://www.kaggle.com/datasets/rtatman/english-word-frequency?resource=download =

In [19]:
# 1.read the table
df = pd.read_csv("unigram_freq.csv", dtype={"word": str, "count": int})

# 2.turn every word into its lemma
def to_lemma(w):
    if pd.isna(w) or w == "":
        return ""
    doc = nlp(w)
    return doc.sentences[0].words[0].lemma

df["lemma"] = df["word"].apply(to_lemma)

# 3.write the full table
df.to_csv("unigram_freq_lemmatized.csv", index=False)

file saved: unigram_freq_lemmatized.csv


In [22]:
#check the data:
df_copy = pd.read_csv('unigram_freq_lemmatized.csv')

In [23]:
df = df_copy
df.head(5)

,word,count,lemma
0,the,1661014682,the
1,of,267040888,of
2,and,112736078,and
3,to,-747921030,to
4,a,491240106,a


In [24]:
#create two new columns with unique lemmas (removing duplicates) and adding up their counts
#group by 'lemma', sum the counts
if "count of unique lemmas" not in df.columns:
    df["count of unique lemmas"] = df.groupby("lemma")["count"].transform("sum")

#keep the first row for every lemma and delete the rest
df.drop_duplicates(subset="lemma", keep="first", inplace=True)

# update the rate that uses the summed count
df["times per million"] = (df["count of unique lemmas"] * 1_000_000 / 1_000_000_000_000).round(2)

#tidy up the index
df.reset_index(drop=True, inplace=True)


In [28]:
df.head(3)

,word,count,lemma,count of unique lemmas,times per million
0,the,1661014682,the,1.703131e+09,1703.13
1,of,267040888,of,5.194060e+08,519.41
2,and,112736078,and,5.540612e+08,554.06


In [29]:
#test 
df.loc[df["lemma"] == "verify", "times per million"]

3558    27.33
Name: times per million, dtype: float64

In [30]:
#this will create the final list that will store only the common words of English with a frequency higher than 0.1 times per million
#We'll use this classification in bands: https://www.oed.com/information/understanding-entries/frequency?tl=true
#cut-off
#keep lemmas with frequency > 0.1 per million

df_most_common_english_lemmas = (df.loc[np.floor(df["times per million"] * 10) / 10 > 0.1,["lemma", "times per million"]].copy().reset_index(drop=True))

In [31]:
df_most_common_english_lemmas.to_excel('Most common English lemmas.xlsx', index=False)

In [32]:
df_most_common_english_lemmas.head(2)

,lemma,times per million
0,the,1703.13
1,of,519.41


In [33]:
len(df_most_common_english_lemmas)

58746